# 1

In [153]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time

URL = 'https://search.naver.com/search.naver'
params = {'query':'날씨'}

response = requests.get(URL, params)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')

message1 = """==============================
날씨 확인 프로그램
=============================="""
message2 = """------------------------------
1. 현재 날씨 및 온도 확인
2. 최저/최고 온도 확인
3. 미세먼지 확인
0. 종료
------------------------------"""
sep1 = '=============================='
sep2 = '------------------------------'


while True:
    ts = time.localtime()
    now = datetime(ts.tm_year, ts.tm_mon, ts.tm_mday, ts.tm_hour, ts.tm_min, ts.tm_sec)
    location = soup.find('h2', attrs={'class':'title'})
    
    print(message1)
    print(f'접속 시간: {now}')
    print(f'접속 위치: {location.get_text()}')
    print(message2)
    
    choice = int(input('번호를 입력하세요.'))
    if choice == 0:
        break
        
    if choice == 1:
        print(f'{sep1} \n현재 날씨 확인\n{sep1}')
        weather = soup.find('div', attrs={'class':'weather_main'}).get_text()
        temp = soup.find('div', attrs={'class':'temperature_text'}).get_text()
        print(f'{sep2} \n{weather} / {temp}\n{sep2}\n')
    elif choice == 2 :
        print(f'{sep1} \n최저/최고 온도 확인\n{sep1}')
        high = soup.find('span', attrs={'class':'lowest'}).get_text()
        low = soup.find('span', attrs={'class':'highest'}).get_text()
        print(f'{sep2} \n최고 기온: {high[4:]}\n최저 기온: {low[4:]}\n{sep2}\n')
    elif choice == 3:
        print(f'{sep1} \n미세먼지 확인\n{sep1}')
        dust = soup.select_one('#main_pack > section.sc_new.cs_weather_new._cs_weather > div._tab_flicking > div.content_wrap > div.open > div:nth-child(1) > div > div.weather_info > div > div.report_card_wrap > ul > li:nth-child(1) > a > span').get_text()
        print(f'{sep2} \n{dust}\n{sep2}\n')
        

날씨 확인 프로그램
접속 시간: 2023-02-01 16:25:59
접속 위치: 강남구 삼성1동
------------------------------
1. 현재 날씨 및 온도 확인
2. 최저/최고 온도 확인
3. 미세먼지 확인
0. 종료
------------------------------
번호를 입력하세요.1
현재 날씨 확인
------------------------------ 
 맑음  /  현재 온도2.8° 
------------------------------

날씨 확인 프로그램
접속 시간: 2023-02-01 16:26:00
접속 위치: 강남구 삼성1동
------------------------------
1. 현재 날씨 및 온도 확인
2. 최저/최고 온도 확인
3. 미세먼지 확인
0. 종료
------------------------------
번호를 입력하세요.0


# 2
- 네이버에서 bts이미지를 검색한 후 나온 결과 중 5개의 이미지를 저장하기 (25점)

• 다음의 처리 단계는 Selenium을 이용한다.
(1) 네이버 홈페이지 요청
(2) 검색 필드에 ‘bts’ 검색어 입력 후 검색
(3) 이미지 검색결과 선택
(4) 최초 검색된 bts 이미지 5장 저장


In [166]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

SCROLL_PAUSE_TIME = 1 # 페이지가 로드될 때까지 기다리는 시간
SEARCH_KEYWORD = 'bts'
IMAGE_EXTRACT_NUM = 5

########################### 네이버 접속 ###########################
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=webdriver.ChromeOptions())
driver.get('https://www.naver.com')


###########################   검색   ###########################
element = driver.find_element(by=By.ID, value='query')
element.send_keys(SEARCH_KEYWORD)
element.submit()


################# 이미지 검색결과 페이지로 이동 #################
driver.find_element(by=By.LINK_TEXT, value='이미지').click()
time.sleep(SCROLL_PAUSE_TIME) # 페이지가 로그될 때까지 기다림


################# 최초 검색된 bts 이미지 추출 #################
imgs = driver.find_elements(By.CSS_SELECTOR, '#main_pack > section.sc_new.sp_nimage._prs_img._imageSearchPC > div > div.photo_group._listGrid > div.photo_tile._grid > div')

# print('이미지 개수: ', len(imgs)) #50개

img_src_list = []

img_cnt = 0
for cnt, img in enumerate(imgs, start=1):
    img.click()
    img_a = driver.find_element(By.CSS_SELECTOR, '#main_pack > section.sc_new.sp_nimage._prs_img._imageSearchPC > div > div.viewer_group._viewerRoot > div > div > div.viewer_flick > div.api_pcpg_wrap > div > div > div.image_area._queryImage > div.image._imageBox > img')
    img_src = img_a.get_attribute('src')

    if not img_src.startswith('http'):
        continue
    
    img_cnt += 1
#     print(f'[{img_cnt}]: {img_src}')
    img_src_list.append(img_src)
    
    if img_cnt == IMAGE_EXTRACT_NUM:break


        
#################### 검색 이미지 파일로 저장 ####################        
import requests
import os

# c드라이브 > temp 폴더를 루트 디렉토리로 잡아 이미지를 저장
path = 'c:/Temp/'
# 시간을 원하는 형식으로 출력
ts = time.localtime() # time stamp
now = f'{ts.tm_year}.{ts.tm_mon}.{ts.tm_mday}.{ts.tm_hour}.{ts.tm_min}.{ts.tm_sec}'
directory = SEARCH_KEYWORD+'/'+now+'/' # python이라는 중간폴더 만든 것
# c:/Temp/python/2023.01.31.14.55.23/
os.chdir(path) #change directory 명령어를 수행하는 함수 # c:/Temp로 디렉토리를 변경하라

if not os.path.exists(directory):
    os.makedirs(directory)

    
file_no = 1
os.chdir(path+directory)


for url in img_src_list:
    extension = url.split('.')[-1] 
    if extension in ['jpg', 'JPG', 'jpeg', 'JPEG', 'png', 'PNG', 'gif', 'GIF']:
        ext = '.'+extension
    else:
        ext = '.jpg'
    file_name = str(file_no) + '-'+ SEARCH_KEYWORD + ext

    res = requests.get(url)
    with open (file_name, 'wb') as f:
        # 디렉토리 정해놨으므로 파일 경로 없이 파일 이름만 명시해도 된다
        f.write(res.content) 
    file_no +=1

driver.close()
print(f'그림 저장 완료! \nTemp의 bts 폴더를 확인하세요')

그림 저장 완료! 
Temp의 bts 폴더를 확인하세요


# 3
- 멜론 사이트 최신곡 검색 (25점)

• 다음 URL을 이용하여 멜론 사이트에서 국내 최신곡 10위까지의 정보(노래제목, 가수, 앨범)를 출력하시오.

• URL : https://www.melon.com/new/index.htm


In [240]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

SCROLL_PAUSE_TIME = 1 # 페이지가 로드될 때까지 기다리는 시간
IMAGE_EXTRACT_NUM = 5

########################### 멜론 접속 ###########################
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=webdriver.ChromeOptions())
driver.get('https://www.melon.com/new/index.htm')


########################### 정보 확인 ###########################
from bs4 import BeautifulSoup
bs = BeautifulSoup(driver.page_source, 'html.parser')


######################## 10위 정보 추출 ########################
rank = bs.find_all('span', attrs={'class':'rank'})
title = bs.find_all('div', attrs={'class':'ellipsis rank01'})
artist = bs.find_all('span', attrs={'class':'checkEllipsis'})
album = bs.find_all('div', attrs={'class':'ellipsis rank03'})

print('순위', '\t\t곡목', '\t\t\t\t가수', '\t\t\t\t앨범', '\n')

for i, r in enumerate(rank):
    rank_no = r.get_text()
    if rank_no == 'NO':
        continue
    else:
        rank_no = int(rank_no)
        if rank_no <= 10:
#             print('%-3d %-25s %20s %30s' %(rank_no, title[i-1].get_text().strip(), artist[i-1].get_text().strip(), album[i-1].get_text().strip()))
            print(f'{rank_no} \t {title[i-1].get_text().strip():22s} \t {artist[i-1].get_text().strip():40s} \t {album[i-1].get_text().strip()}')

순위 		곡목 				가수 				앨범 

1 	 애모                     	 민수현, 신명근, 정다한, 춘길, 황영웅                   	 불타는 트롯맨 디너쇼 미션 PART 1
2 	 Fuck Around (Feat. Gist) 	 NSW yoon                                 	 In My Mood
3 	 하나뿐인 운명처럼              	 MZ수채학 (잠골버스2기)                           	 하나뿐인 운명처럼
4 	 걱정 붙들어 매               	 정미애                                      	 걱정 붙들어 매
5 	 마음에 닿기를                	 노르웨이 숲                                   	 마음에 닿기를
6 	 PULL UP                	 VIVIZ (비비지)                              	 The 3rd Mini Album 'VarioUS'
7 	 Trust Yourself         	 Balming Tiger                            	 Trust Yourself
8 	 Love is very difficult 	 Pa!nter                                  	 Love is very difficult
9 	 Passing Tourists       	 Azikazin Magic World                     	 Passing Tourists
10 	 눈 빛 (snow light)       	 홍대광                                      	 눈 빛 (snow light)


# 4
- 4. 대문자 시작 단어 찾기 (10점)

• 다음 문자열에서 정규 표현식을 이용하여 대문자로 시작하는 단어를 찾아 출력하시오.

s = 'Edit the Expression & Text to see matches. Roll over matches or the expression for details. PCRE & JavaScript flavors of RegEx are supported. Validate your expression with Tests mode.'

- 출력결과: ['Edit', 'Expression', 'Text', 'Roll', 'PCRE', 'JavaScript', 'RegEx', 'Validate', 'Tests']

In [57]:
import re

s = 'Edit the Expression & Text to see matches. Roll over matches or the expression for details. PCRE & JavaScript flavors of RegEx are supported. Validate your expression with Tests mode.'

p = re.compile(r'\b[A-Z]\w+\b')
result = p.findall(s)
print(f'출력결과: {result}')

출력결과: ['Edit', 'Expression', 'Text', 'Roll', 'PCRE', 'JavaScript', 'RegEx', 'Validate', 'Tests']


# 5

- 5. 이메일 주소 추출하기 (10점)

• 다음 문자열에서 정규 표현식을 이용하여 이메일 주소를 찾아 출력하시오.

s = 'Ryan has sent an invoice email to john.d@yahoo.com by using his email id ryan.arjun@gmail.com and he also shared a copy to his boss rosy.gray@amazon.co.uk on the cc part.'

- 출력결과: ['john.d@yahoo.com', 'ryan.arjun@gmail.com', 'rosy.gray@amazon.co.uk']

In [58]:
import re

s = 'Ryan has sent an invoice email to john.d@yahoo.com by using his email id ryan.arjun@gmail.com and he also shared a copy to his boss rosy.gray@amazon.co.uk on the cc part.'

p = re.compile(r'\b[\w.]+[@][\w.]+\b')
result = p.findall(s)
print(f'출력결과: {result}')

출력결과: ['john.d@yahoo.com', 'ryan.arjun@gmail.com', 'rosy.gray@amazon.co.uk']
